In [1]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from BigDFT.Database import Molecules
from futile.Utils import write
import numpy as np
import matplotlib.pyplot as plt
import os, sys, yaml
sys.path.insert(0,'../')
import StatPol as SP, Routines as R

basepath = os.getcwd()

AuToA = 0.5291772085**3

Routines basepath : /home/marco/Data/RICERCA/LINEAR RESPONSE/Statical polarizability/Analysis/Calculations and data analysis


## Data analysis

In [2]:
nsp_results = yaml.load(open('nsp_results.yaml'))
#nsp_results

In [4]:
# Build the list of molecules for which the 'results' key is provided
computed_mol = []
for mol,data in nsp_results.iteritems():
    if 'results' in data:
        computed_mol.append(mol)
computed_mol.sort()
print len(computed_mol)
computed_mol

25


['AlF',
 'Ar',
 'BF',
 'BH2Cl',
 'BH2F',
 'BH3',
 'BHF2',
 'BeH2',
 'C2H2',
 'C2H4',
 'CH2BH',
 'CH3BH2',
 'CH3Cl',
 'CH3F',
 'CH3NH2',
 'CH3OH',
 'CH3SH',
 'CH4',
 'CO',
 'CO2',
 'H2O',
 'HCl',
 'Mg',
 'Mg2',
 'N2']

Several type of data analysis can be done. For instance

We choose a xc and compute (for all the molecule for which results have been computed) the difference the ref_results and the ones obtained with the allowed psp's

In [24]:
xc = 'pbe'
data = nsp_results['CO']
ref = data['ref_results'][xc]
ref

[1.856, 1.856, 2.363]

In [30]:
# extract the all the results with xc
psps = []
for s in data['study']:
    if s[0]==xc:
        psps.append(s[1])
print psps

['hgh-k', 'nlcc']


In [42]:
# extract the converged results of alpha associate to each allowed study

for psp in psps:
    results = data['results'][xc,psp]['rmult_conv']
    conv_val = results['converged_value']
    alpha = results['results'][conv_val]
    print alpha

[[ 1.25060100e+01 -1.50890000e-05 -2.85740000e-05]
 [-1.50890000e-05  1.25060100e+01 -2.85740000e-05]
 [-2.70000000e-05 -2.70000000e-05  1.58900495e+01]]
[[ 1.25368250e+01 -1.58955000e-05 -2.55350000e-05]
 [-1.58955000e-05  1.25368250e+01 -2.55350000e-05]
 [-4.40000000e-05 -4.40000000e-05  1.58737875e+01]]


In [13]:
for mol in computed_mol:
    data = nsp_results[mol]['results']

In [14]:
data

{('lda',
  'hgh-k'): {'field_conv': {'converged': True,
   'converged_value': 0.01,
   'label': 'field_int',
   'results': {0.0005: None,
    0.001: None,
    0.005: matrix([[ 1.0510535e+01, -1.2548000e-04, -9.7510000e-05],
            [-1.2548000e-04,  1.0510535e+01, -9.7510000e-05],
            [-4.7960000e-05, -4.7960000e-05,  1.5078189e+01]]),
    0.01: matrix([[ 1.051914e+01, -1.236400e-04, -9.836500e-05],
            [-1.236400e-04,  1.051914e+01, -9.836500e-05],
            [-6.570000e-05, -6.570000e-05,  1.509147e+01]])},
   'values': [0.01, 0.005, 0.001, 0.0005]}, 'gs_conv': {'converged': True,
   'converged_value': 5.0,
   'label': 'rmult',
   'results': {3.0: -19.839933508992686,
    4.0: -19.885289313084087,
    5.0: -19.888866392556345,
    6.0: -19.889161787439456,
    7.0: None,
    8.0: None,
    9.0: None,
    10.0: None,
    11.0: None},
   'values': [3.0,
    4.0,
    5.0,
    6.0,
    7.0,
    8.0,
    9.0,
    10.0,
    11.0]}, 'rmult_conv': {'converged': True,
   

In [ ]:
################################################################################################

We build a table with the (converged) values of the average polarizability for all the studies 

In [5]:
AuToA = 0.5291772085**3
for mol,val in set.iteritems():
    for par in val:
        f_conv = val[par]['field_conv']['converged_value']
        r_conv = val[par]['rmult_conv']['converged_value']
        alpha = val[par]['rmult_conv']['results'][r_conv]
        alpha_avg = SP.eval_alpha_avg(alpha)
        print mol, par, 'alpha(AU) =',"{:5.4f}".format(alpha_avg),\
        'alpha(A) =',"{:4.3f}".format(alpha_avg*AuToA), 'r_conv = ', r_conv, 'f_conv = ', f_conv
    

Mg ('lda', 'hgh-k') alpha(AU) = 71.8717 alpha(A) = 10.650 r_conv =  8.0 f_conv =  0.005
Mg ('pbe', 'hgh-k') alpha(AU) = 73.4948 alpha(A) = 10.891 r_conv =  8.0 f_conv =  0.005
CO ('lda', 'hgh-k') alpha(AU) = 13.7103 alpha(A) = 2.032 r_conv =  6.0 f_conv =  0.01
CO ('pbe', 'hgh-k') alpha(AU) = 13.6340 alpha(A) = 2.020 r_conv =  6.0 f_conv =  0.01
Ne ('lda', 'hgh-k') alpha(AU) = 3.0331 alpha(A) = 0.449 r_conv =  7.0 f_conv =  0.01
Ne ('pbe', 'hgh-k') alpha(AU) = 3.0552 alpha(A) = 0.453 r_conv =  7.0 f_conv =  0.01
NaCl ('lda', 'hgh-k') alpha(AU) = 35.5620 alpha(A) = 5.270 r_conv =  7.0 f_conv =  0.001
NaCl ('pbe', 'hgh-k') alpha(AU) = 37.1554 alpha(A) = 5.506 r_conv =  7.0 f_conv =  0.001
Mg2 ('lda', 'hgh-k') alpha(AU) = 158.0223 alpha(A) = 23.416 r_conv =  7.0 f_conv =  0.001
Mg2 ('pbe', 'hgh-k') alpha(AU) = 163.0076 alpha(A) = 24.155 r_conv =  8.0 f_conv =  0.001
H2O ('lda', 'hgh-k') alpha(AU) = 10.5996 alpha(A) = 1.571 r_conv =  6.0 f_conv =  0.01
H2O ('pbe', 'hgh-k') alpha(AU) = 10.5